# Carregar models e inferencia com Hugging Face

Neste projeto, vamos explorar a biblioteca de transformadores Hugging Face para várias tarefas de processamento de linguagem natural (NLP). Vamos realizar a classificação de texto e a geração de texto usando modelos pré-treinados como `DistilBERT` e `GPT-2` sem usar a função `pipeline()`, entendendo as etapas envolvidas no carregamento de modelos, tokenização de entrada, execução de inferência e processamento de saídas. Vamos entender a simplicidade e a eficiência de usar a função `pipeline()` para realizar as mesmas tarefas com o mínimo de código. A função `pipeline()` simplifica o processo, tornando mais fácil e rápido implementar soluções de NLP.

### Objetivos

- Aprenda a configurar e usar a biblioteca Hugging Face `transformers`.
- Execute a classificação de texto e geração de texto usando modelos DistilBERT e GPT-2 sem `pipeline()`.
- Entenda e utilize a função `pipeline()` para simplificar várias tarefas de PNL.
- Compare a facilidade de usar modelos diretamente versus usar a função `pipeline()`.

### Preparar setup

In [2]:
def warn(*args, **kwargs):
    pass
    
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import importlib.util
import subprocess
import sys

def check_and_install(package, pip_name=None):
    if pip_name is None:
        pip_name = package
    spec = importlib.util.find_spec(package)
    if spec is None:
        print(f"{package} não está instalado. Instalando...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])
    else:
        print(f"{package} já está instalado.")

In [3]:
check_and_install('torch')
check_and_install('transformers')

torch já está instalado.
transformers já está instalado.


### Carregar bibliotecas

In [4]:
from transformers import pipeline
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

### Classificação de texto com DistilBERT
Inicializar um tokenizador e um modelo para análise de sentimento usando o DistilBERT ajustado no conjunto de dados SST-2. Essa configuração é útil para tarefas em que você precisa classificar rapidamente o sentimento de um pedaço de texto com um modelo de transformador eficiente e pré-treinado.

In [5]:
# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

### Pre processar texto de input
Tokenizar o texto de input e convertar para o formato adequado para o modelo

In [7]:
# Sample text
text = """
Congratulation! You've won a free ticket to the Bahamas.
Reply WIN to claim.
"""
text

"\nCongratulation! You've won a free ticket to the Bahamas.\nReply WIN to claim.\n"

In [9]:
# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101, 26478,  8609,  9513,   999,  2017,  1005,  2310,  2180,  1037,
          2489,  7281,  2000,  1996, 17094,  1012,  7514,  2663,  2000,  4366,
          1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Os ids de token são os índices de token ```attention_mask``` são essenciais para processar corretamente sequências preenchidas, garantindo computação eficiente e mantendo o desempenho do modelo. Mesmo quando nenhum token é explicitamente mascarado, ele ajuda o modelo a diferenciar entre conteúdo real e preenchimento, o que é crítico para o processamento preciso e eficiente de dados de entrada.

### Executar inferência
O gerenciador de contexto `torch.no_grad()` é usado para desabilitar o cálculo de gradiente.
Isso reduz o consumo de memória e acelera a computação, pois gradientes não são necessários para inferência (ou seja, quando você não está treinando o modelo). A sintaxe **inputs é usada para descompactar um dicionário de argumentos de palavras-chave em Python. No contexto do model(**inputs):

In [10]:
# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

In [11]:
# Another method is input_ids, and attention_mask is their own parameter
#model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

#### Obtenha os logits
Os logits são as previsões brutas e não normalizadas do modelo. Vamos extrair os logits das saídas do modelo para executar processamento posterior, como determinar a classe prevista ou calcular probabilidades.

In [12]:
logits = outputs.logits
logits.shape

torch.Size([1, 2])

### Pós-processar a saída
Converter os logits em probabilidades e obter a classe prevista:

In [13]:
# Convert logits to probabilities
probs = torch.softmax(logits, dim=-1)

# Get the predicted class
predicted_class = torch.argmax(probs, dim=-1)

In [14]:
# Map the predicted class to the label
labels = ["NEGATIVE", "POSITIVE"]
predicted_label = labels[predicted_class]

print(f"Predicted label: {predicted_label}")

Predicted label: POSITIVE


### Geração de texto com GPT-2

### Carregar tokenizador
Carregar o tokenizador pre-treinado GPT-2. O tokenizador pe responsável por converter texto em tokens que o modelo consegue entender

In [15]:
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Carregar o modelo GPT-2 pré-treinado com um head de modelagem de linguagem. O modelo gera texto com base nos tokens de entrada.

In [16]:
# Load the tokenizer and model
model = GPT2LMHeadModel.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Pré-processe o texto de entrada
Tokenize o texto de entrada e converta-o para um formato adequado ao modelo, como antes de ter os índices de token, ou seja, entradas.

In [17]:
# Prompt
prompt = "Once upon a time"

In [18]:
# Tokenize the input text
inputs = tokenizer(prompt, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[7454, 2402,  257,  640]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


### Executar inferência
Gerar texto usando o modelo

```inputs:``` IDs de token de entrada do tokenizador

```attention_mask:``` Máscara indicando quais tokens atender

```pad_token_id:```ID de token de preenchimento definido para o ID de token de fim de sequência

```max_length:``` Comprimento máximo das sequências geradas

```num_return_sequence:``` Número de sequências a serem geradas

In [19]:
# Generate text
output_ids = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    max_length=50,
    num_return_sequences=1
)

## Pós-processar a saída
Decodifique os tokens gerados para obter o texto:

In [20]:
# Decode the generated text
generated_text = tokenizer.decode(output_ids[0],
                                  skip_special_tokens=True)

In [21]:
print(generated_text)

Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a


### Função `pipeline()` do Hugging Face

A função `pipeline()` da biblioteca `transformers` do Hugging Face é uma API de alto nível projetada para simplificar o uso de modelos pré-treinados para várias tarefas de processamento de linguagem natural (NLP). Ela abstrai as complexidades do carregamento de modelos, tokenização, inferência e pós-processamento, permitindo que os usuários realizem tarefas NLP complexas com apenas algumas linhas de código.

#### Definição

```python
transformers.pipeline(
    task: str,
    model: Optional = None,
    config: Optional = None,
    tokenizer: Optional = None,
    feature_extractor: Optional = None,
    framework: Optional = None,
    revision: str = 'main',
    use_fast: bool = True,
    model_kwargs: Dict[str, Any] = None,
    **kwargs
)
```

## Parâmetros

- **task**: `str`
- A tarefa a ser executada, como "classificação de texto", "geração de texto", "resposta a perguntas", etc.
- Exemplo: `"text-classification"`

- **model**: `Opcional`
- O modelo a ser usado. Pode ser uma string (identificador de modelo do hub de modelos Hugging Face), um caminho para um diretório contendo arquivos de modelo ou uma instância de modelo pré-carregada.
- Exemplo: `"distilbert-base-uncased-finetuned-sst-2-english"`

- **config**: `Opcional`
- A configuração a ser usada. Pode ser uma string, um caminho para um diretório ou um objeto de configuração pré-carregado.
- Exemplo: `{"output_attentions": True}`

- **tokenizer**: `Opcional`
- O tokenizador a ser usado. Pode ser uma string, um caminho para um diretório ou uma instância de tokenizador pré-carregada.
- Exemplo: `"bert-base-uncased"`

- **feature_extractor**: `Opcional`
- O extrator de recursos a ser usado para tarefas que o exigem (por exemplo, processamento de imagens).
- Exemplo: `"facebook/detectron2"`

- **framework**: `Opcional`
- O framework a ser usado, seja `"pt"` para PyTorch ou `"tf"` para TensorFlow. Se não for especificado, será inferido.
- Exemplo: `"pt"`

- **revision**: `str`, padrão `'main'`
- A versão específica do modelo a ser usada (branch, tag ou hash de commit).
- Exemplo: `"v1.0"`

- **use_fast**: `bool`, padrão `True`
- Se deve usar a versão rápida do tokenizador, se disponível.
- Exemplo: `True`

- **model_kwargs**: `Dict[str, Any]`, padrão `None`
- Argumentos de palavra-chave adicionais passados ​​para o modelo durante a inicialização.
- Exemplo: `{"output_hidden_states": True}`

- **kwargs**: `Any`
- Argumentos de palavra-chave adicionais passados ​​para os componentes do pipeline.

## Tipos de tarefa

A função `pipeline()` suporta uma ampla gama de tarefas de PNL. Aqui estão algumas das tarefas comuns:

1. **Classificação de texto**: `text-classification`
- **Objetivo**: Classificar texto em categorias predefinidas.
- **Casos de uso**: Análise de sentimento, detecção de spam, classificação de tópicos.

2. **Geração de texto**: `text-generation`
- **Objetivo**: Gerar texto coerente com base em um prompt fornecido.
- **Casos de uso**: Escrita criativa, geração de diálogo, conclusão de história.

3. **Resposta a perguntas**: `question-answering`
- **Objetivo**: Responder a perguntas com base em um contexto fornecido.
- **Casos de uso**: Construir sistemas de perguntas e respostas, recuperação de informações de documentos.

4. **Reconhecimento de Entidade Nomeada (NER)**: `ner` (ou `token-classification`)
- **Objetivo**: Identificar e classificar entidades nomeadas (como pessoas, organizações, locais) no texto.
- **Casos de Uso**: Extrair informações estruturadas de texto não estruturado.

5. **Resumo**: `summarization`
- **Objetivo**: Resumir longos trechos de texto em resumos mais curtos e coerentes.
- **Casos de Uso**: Resumo de documentos, resumo de notícias.

6. **Tradução**: `translation_xx_to_yy` (por exemplo, `translation_en_to_fr`)
- **Objetivo**: Traduzir texto de um idioma para outro.
- **Casos de Uso**: Tradução de idiomas, aplicativos multilíngues.

7. **Fill-Mask**: `fill-mask`
- **Propósito**: Prever palavras mascaradas em uma frase (útil para modelagem de linguagem mascarada).
- **Casos de uso**: Tarefas de modelagem de linguagem, compreensão de previsões de modelo.

8. **Classificação Zero-Shot**: `zero-shot-classification`
- **Propósito**: Classificar texto em categorias sem precisar de dados de treinamento para essas categorias.
- **Casos de uso**: Tarefas de classificação flexíveis e adaptáveis.

9. **Extração de recursos**: `feature-extraction`
- **Propósito**: Extrair recursos de estado ocultos do texto.
- **Casos de uso**: Tarefas posteriores que exigem representações de texto, como agrupamento, similaridade ou treinamento de modelo personalizado adicional.

### Exemplo 1: Classificação de texto usando `pipeline()`

Neste exemplo, você usará a função `pipeline()` para executar a classificação de texto. Você carregará um modelo de classificação de texto pré-treinado e o usará para classificar um texto de amostra.

#### Carregue o modelo de classificação de texto:
Inicializamos o pipeline para a tarefa `text-classification`, especificando o modelo `"distilbert-base-uncased-finetuned-sst-2-english"`. Este modelo é ajustado para análise de sentimento.

#### Classifique o texto de amostra:
Usamos o classificador para classificar um texto de amostra: "Congratulations! You've won a free ticket to the Bahamas. Reply WIN to claim." A função `classifier` retorna o resultado da classificação, que é então impresso.

In [26]:
# Load a general text classification model
classifier = pipeline("text-classification",
                      model="distilbert-base-uncased-finetuned-sst-2-english",
                     framework='pt')

In [27]:
# Classify a sample text
result = classifier("Congratulations! You've won a free ticket to the Bahamas. Reply WIN to claim.")
print(result)

[{'label': 'POSITIVE', 'score': 0.9997586607933044}]


### Exemplo 2: Detecção de idioma usando `pipeline()`

Neste exemplo, você usará a função `pipeline()` para executar a detecção de idioma. Você carregará um modelo de detecção de idioma pré-treinado e o usará para identificar o idioma de um texto de amostra.

#### Carregue o modelo de detecção de idioma:
Inicializamos o pipeline para a tarefa `text-classification`, especificando o modelo `"papluca/xlm-roberta-base-language-detection"`. Este modelo é ajustado para detecção de idioma.

#### Classifique o texto de amostra:
Usamos o classificador para detectar o idioma de um texto de amostra: "Bonjour, comment ça va?" A função `classifier` retorna o resultado da classificação, que é então impresso.

In [28]:
classifier = pipeline("text-classification", 
                      model="papluca/xlm-roberta-base-language-detection",
                     framework='pt')

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [29]:
result = classifier("Bonjour, comment ça va?")
print(result)

[{'label': 'fr', 'score': 0.9934879541397095}]


### Exemplo 3: Geração de texto usando `pipeline()`

Neste exemplo, você usará a função `pipeline()` para executar a geração de texto. Você carregará um modelo de geração de texto pré-treinado e o usará para gerar texto com base em um prompt fornecido.

#### Inicialize o modelo de geração de texto:
Inicializamos o pipeline para a tarefa `text-generation`, especificando o modelo `"gpt2"`. GPT-2 é um modelo bem conhecido para tarefas de geração de texto.

In [31]:
# Initialize the text generation pipeline with GPT-2
generator = pipeline("text-generation", model="gpt2", framework='pt')

#### Gerar texto com base em um prompt fornecido:
Usamos o gerador para gerar texto com base em um prompt: "Once upon a time". Vamos especificar `max_length=50`, `truncation=True` para limitar o texto gerado a 50 tokens e `num_return_sequences=1` para gerar uma sequência. A função `generator` retorna o texto gerado, que é então impresso.

In [32]:
# Generate text based on a given prompt
prompt = "Once upon a time"

In [34]:
result = generator(prompt, 
                   max_length=50,
                   num_return_sequences=1,
                   truncation=True)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [36]:
# Print the generated text
print(result[0]['generated_text'])

Once upon a time it was expected that the entire country would start the debate, but just six days later, the President decided that time had actually run out. The American people have done enough. They have demanded change. They have called for a new


### Exemplo 4: Geração de texto usando T5 com `pipeline()`

Neste exemplo, você usará a função `pipeline()` para executar a geração de texto para texto com o modelo T5. Você carregará um modelo T5 pré-treinado e o usará para traduzir uma frase do inglês para o francês com base em um prompt fornecido.

#### Inicialize o modelo de geração de texto:
Inicializamos o pipeline para a tarefa `text2text-generation, especificando o modelo "t5-small". O T5 é um modelo versátil que pode executar várias tarefas de geração de texto para texto, incluindo tradução.

In [37]:
# Initialize the text generation pipeline with T5
generator = pipeline("text2text-generation", model="t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

#### Gerar texto com base em um prompt fornecido:
Usamos o gerador para traduzir uma frase do inglês para o francês com base no prompt: "translate English to French: How are you?". Vamos especificar `max_length=50` para limitar o texto gerado a 50 tokens e `num_return_sequences=1` para gerar uma sequência. A função `generator` retorna o texto traduzido, que é então impresso.

In [38]:
# Generate text based on a given prompt
prompt = "translate English to French: How are you?"

In [39]:
result = generator(prompt,
                   max_length=50,
                   num_return_sequences=1)

In [40]:
# Print the genrated text
print(result[0]['generated_text'])

Comment êtes-vous?


## Benefícios de usar `pipeline()`

- **Código Boilerplate Reduzido**: Simplifica o código necessário para executar tarefas de PNL.
- **Legibilidade Aprimorada**: Torna o código mais legível e expressivo.
- **Eficiência de Tempo**: Economiza tempo ao manipular o carregamento de modelos, tokenização, inferência e pós-processamento automaticamente.
- **API Consistente**: Fornece uma API consistente em diferentes tarefas, permitindo fácil experimentação e prototipagem rápida.
- **Manipulação Automática de Framework**: Manipula automaticamente o framework subjacente (TensorFlow ou PyTorch).

## Quando usar `pipeline()`

- **Prototipagem rápida**: quando você precisa prototipar rapidamente um aplicativo NLP ou experimentar diferentes modelos.
- **Tarefas simples**: ao executar tarefas NLP simples ou comuns que são bem suportadas pela função `pipeline()`.
- **Implantação**: ao implantar modelos NLP em ambientes onde simplicidade e facilidade de uso são cruciais.

## Quando evitar `pipeline()`

- **Tarefas personalizadas**: quando você precisa executar tarefas altamente personalizadas que não são bem suportadas pela função `pipeline()`.
- **Otimização de desempenho**: quando você precisa de controle refinado sobre o modelo e o processo de tokenização para otimização de desempenho ou casos de uso específicos.

### Exercício: tarefa de máscara de preenchimento usando BERT com `pipeline()`

Neste exercício, você usará a função `pipeline()` para executar uma tarefa de máscara de preenchimento usando o modelo BERT. Você carregará um modelo BERT pré-treinado e o usará para prever a palavra mascarada em uma determinada frase.

In [41]:
# Initialize the fill-mask pipeline with BERT
fill_mask = pipeline("fill-mask", model="bert-base-uncased")

# Generate text by filling in the masked token
prompt = "The capital of France is [MASK]."
result = fill_mask(prompt)

# Print the generated text
print(result)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[{'score': 0.416788786649704, 'token': 3000, 'token_str': 'paris', 'sequence': 'the capital of france is paris.'}, {'score': 0.07141651213169098, 'token': 22479, 'token_str': 'lille', 'sequence': 'the capital of france is lille.'}, {'score': 0.06339280307292938, 'token': 10241, 'token_str': 'lyon', 'sequence': 'the capital of france is lyon.'}, {'score': 0.04444750398397446, 'token': 16766, 'token_str': 'marseille', 'sequence': 'the capital of france is marseille.'}, {'score': 0.03029724210500717, 'token': 7562, 'token_str': 'tours', 'sequence': 'the capital of france is tours.'}]
